# Reporting Basics - 2

To construct necessary reporting tables, we will need to use certain SAS tools to build reports. We will need 
* PROC FREQ
* PROC SUMMARY
* PROC REPORT
* PROC TEMPLATE
* PROC FORMAT

## PROC REPORT

The FREQ procedure produces one-way to n-way frequency and contingency (crosstabulation) tables.
* It calculates the number of observations and the percentages identified by a 

In [ ]:
data x;
	set sashelp.cars (where = ((make = "Acura" or make = "Mercedes-Benz") and (type = "SUV" or type = "Sedan")));
	id + 1; /* <-- Generate an ID */
    if make = "Acura" then makeid = 0;
    else if make = "Mercedes-Benz" then makeid = 1;
    output;
    makeid =2;
    output;
run;

proc sort; 
	by makeid;
run;

proc freq noprint;  
	by makeid; /* <-- Execute PROC FREQ for each make. */ 
	table type / out = out_1; /* <-- The variable I want to tabulate is "type". */
run;

data out_2;
	set out_1;
	_n = put(count,5.0)||"("||put(percent,5.1)||")";
	drop count percent;
run;


proc sort data = out_2;
	by type;
run;

proc transpose data = out_2 out = out_3 prefix = make;
	by type;
	id makeid;
	var _n;
run;

data out_3;
    length type $40 row_name $40;
	set out_3 (drop = _Name_);
	if type = "SUV" then row_name = "    SUV";
	else if type = "Sedan" then row_name = "    Sedan";
run;

data z;
    length row_name $30;
	row_name = "Type [n(%)]";
run;

data out_3 (drop = type);
	set z out_3;
run;

data freq;
    set out_3;
    group = 1;
run;

In [ ]:
/* Another Table */
data x;
	set sashelp.cars (where = ((make = "Acura" or make = "Mercedes-Benz") and (type = "SUV" or type = "Sedan")));
	id + 1; /* <-- Generate an ID */
    if make = "Acura" then makeid = 0;
    else if make = "Mercedes-Benz" then makeid = 1;
    output;
    makeid =2;
    output;
run;

proc sort; 
	by makeid;
run;

proc summary;
    by makeid;
    var msrp;
    output out = out_1 N = _N mean = _mean std = _std median = _median min = _min max = _max;
run;

data out_1 (drop = _:);
    length N $40 mean_std $40 median $40 min_max $40;
	set out_1 (drop = _t: _f:);
	
	N = put(_N,6.);
	mean_std = put(_mean,6.1)||"("||put(_std,6.2)||")";
	median = put(_median,6.1);
	min_max = put(_min,6.)||", "||put(_max,6.);
run;

proc transpose data = out_1 out = out_2 prefix = make;
	id makeid;
	var N mean_std median min_max;
run;

data out_3 (drop = _Name_);
	length row_name $40;
	set out_2;
	if _name_ = "N" then row_name = "    N";
	else if _name_ = "mean_std" then row_name = "    Mean(SD)";
	else if _name_ = "median" then row_name = "    Median";
	else if _name_ = "min_max" then row_name = "    Min, Max";
run;

data z;
	length row_name $40;
	row_name = "MSRP ($)";
run;

data out_3;
	set z out_3;
run;

data sum_stats;
	set out_3;
	group = 2;
run;

In [ ]:
/* Put 2 tables in one */
data out;
    length make0 $40 make1 $40 make2 $40;
    set freq sum_stats;
run;

This is more like a PROC PRINT. Let's see what PROC REPORT can do. 

### Order the columns
We can order the columns. 

In [ ]:
proc report; 
    column(group row_name make1 make0 make2)
run;

/*or*/

proc report; 
    column(row_name make0 make1 group make2)
run;

### Column Names

In [ ]:
proc report split="*"; /* The SPLIT option which can allow column names to break into two rows */
    column(group row_name make0 make1 make2);
    define group / group noprint;
    define row_name / '' width = 80 style=[asis=on];
    define make0 / 'Acura*(N=6)';
	define make1 / 'Mersedes-Benz*(N=18)';
	define make2 / 'Overall*(N=24)';
run;

### Headers and Footnotes

In [ ]:
proc report split="*"; 
    column(group row_name make0 make1 make2);
    define group / group noprint;
    define row_name / '' width = 80 style=[asis=on];
    define make0 / 'Acura*(N=6)';
	define make1 / 'Mersedes-Benz*(N=18)';
	define make2 / 'Overall*(N=24)';
	
	compute before _page_;
	line "";
	line @10 'xx.xx.xx.xx Subject Cars';
	line @20 'Acura vs Benz';
	line "";
	endcomp;
	
	compute after;
	line @4 "footnotes 1";
	line @4 "footnotes 2";
	line @4 "footnotes 3";
	endcomp;
run;

### Column Style

In [ ]:
proc report split="*"; 
    column(group row_name make0 make1 make2);
    define group / group noprint;
    define row_name / '' width = 80 style=[asis=on];
    define make0 / 'Acura*(N=6)';
	define make1 / 'Mersedes-Benz*(N=18)';
	define make2 / 'Overall*(N=24)';
	
	compute before _page_;
	line "";
	line @10 'xx.xx.xx.xx Subject Cars';
	line @20 'Acura vs Benz';
	line "";
	endcomp;
	
	compute after;
	line @4 "footnotes 1";
	line @4 "footnotes 2";
	line @4 "footnotes 3";
	endcomp;
run;

### Header Style

In [ ]:
proc report split="*"; 
    column(group row_name make0 make1 make2);
    define group / group noprint;
    define row_name / '' width = 80 style=[asis=on];
    define make0 / 'Acura*(N=6)' style(header)=Header{background=red};;
	define make1 / 'Mersedes-Benz*(N=18)' style(header)=Header{background=yellow};;
	define make2 / 'Overall*(N=24)' style(header)=Header{background=blue};;
	
	compute before _page_;
	line "";
	line @10 'xx.xx.xx.xx Subject Cars';
	line @20 'Acura vs Benz';
	line "";
	endcomp;
	
	compute after;
	line @4 "footnotes 1";
	line @4 "footnotes 2";
	line @4 "footnotes 3";
	endcomp;
run;